# Implementing a Route Planner
In this project you will use A\* search to implement a "Google-maps" style route planning algorithm.

## The Map

In [242]:
# Run this cell first!

# Libraries
from helpers import Map, load_map_10, load_map_40, show_map
import math

# Reloads modules before executing user code.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Map Basics

In [243]:
# Map Basics
# Loads & Visualizes Map_10
map_10 = load_map_10() #assigns oad_map_10() to variable map_10
show_map(map_10) #Shows a disconnected network of 10 intersections.

The map above (run the code cell if you don't see it) shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. This map is quite literal in its expression of distance and connectivity. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [244]:
#Shows the intersection point as a key value and the intersection coordinates as values.
map_10.intersections

{0: (0.7798606835438107, 0.6922727646627362),
 1: (0.7647837074641568, 0.3252670836724646),
 2: (0.7155217893995438, 0.20026498027300055),
 3: (0.7076566826610747, 0.3278339270610988),
 4: (0.8325506249953353, 0.02310946309985762),
 5: (0.49016747075266875, 0.5464878695400415),
 6: (0.8820353070895344, 0.6791919587749445),
 7: (0.46247219371675075, 0.6258061621642713),
 8: (0.11622158839385677, 0.11236327488812581),
 9: (0.1285377678230034, 0.3285840695698353)}

**Roads**

The `roads` property is a list where `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [245]:
# Shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 

[7, 6, 5]

In [246]:
# Shows the full connectivity of the map
map_10.roads

[[7, 6, 5],
 [4, 3, 2],
 [1, 4, 3],
 [1, 2, 5, 4],
 [1, 2, 3],
 [0, 3, 7],
 [0],
 [0, 5],
 [9],
 [8]]

In [247]:
# Loads & Visualizes Map_40
map_40 = load_map_40() 
show_map(map_40) #shows a disconnected network of 40 intersections.

### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [248]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5,16,37,12,34])

## The Algorithm
### Writing your algorithm
The algorithm written will be responsible for generating a `path` like the one passed into `show_map` above. In fact, when called with the same map, start and goal, as above you algorithm should produce the path `[5, 16, 37, 12, 34]`. However you must complete several methods before it will work.

```bash
> PathPlanner(map_40, 5, 34).path
[5, 16, 37, 12, 34]
```

### PathPlanner class

The below class is already partly implemented for you - you will implement additional functions that will also get included within this class further below.

Let's very briefly walk through each part below.

`__init__` - We initialize our path planner with a map, M, and typically a start and goal node. If either of these are `None`, the rest of the variables here are also set to none. If you don't have both a start and a goal, there's no path to plan! The rest of these variables come from functions you will soon implement. 
- `closedSet` includes any explored/visited nodes. 
- `openSet` are any nodes on our frontier for potential future exploration. 
- `cameFrom` will hold the previous node that best reaches a given node
- `gScore` is the `g` in our `f = g + h` equation, or the actual cost to reach our current node
- `fScore` is the combination of `g` and `h`, i.e. the `gScore` plus a heuristic; total cost to reach the goal
- `path` comes from the `run_search` function, which is already built for you.

`reconstruct_path` - This function just rebuilds the path after search is run, going from the goal node backwards using each node's `cameFrom` information.

`_reset` - Resets *most* of our initialized variables for PathPlanner. This *does not* reset the map, start or goal variables, for reasons which you may notice later, depending on your implementation.

`run_search` - This does a lot of the legwork to run search once you've implemented everything else below. First, it checks whether the map, goal and start have been added to the class. Then, it will also check if the other variables, other than `path` are initialized (note that these are only needed to be re-run if the goal or start were not originally given when initializing the class, based on what we discussed above for `__init__`.

From here, we use a function you will implement, `is_open_empty`, to check that there are still nodes to explore (you'll need to make sure to feed `openSet` the start node to make sure the algorithm doesn't immediately think there is nothing to open!). If we're at our goal, we reconstruct the path. If not, we move our current node from the frontier (`openSet`) and into explored (`closedSet`). Then, we check out the neighbors of the current node, check out their costs, and plan our next move.

This is the main idea behind A*, but none of it is going to work until you implement all the relevant parts, which will be included below after the class code.

In [249]:
# Do not change this cell
# When you write your methods correctly this cell will execute
# without problems
class PathPlanner():
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        self.closedSet = self.create_closedSet() if goal != None and start != None else None
        self.openSet = self.create_openSet() if goal != None and start != None else None
        self.cameFrom = self.create_cameFrom() if goal != None and start != None else None
        self.gScore = self.create_gScore() if goal != None and start != None else None
        self.fScore = self.create_fScore() if goal != None and start != None else None
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None
    
    def reconstruct_path(self, current):
        """ Reconstructs Path after Search """
#         total_path = [current]
#         while current in self.cameFrom.keys():
#             current = self.cameFrom[current]
#             total_path.append(current)
        total_path = self.cameFrom[current]
        return total_path
    
    def _reset(self):
        """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
        self.closedSet = None
        self.openSet = None
        self.cameFrom = None
        self.gScore = None
        self.fScore = None
        self.path = self.run_search() if self.map and self.start and self.goal else None

    def run_search(self):
        """ """
        if self.map == None:
            raise(ValueError, "Must create map before running search. Try running PathPlanner.set_map(start_node)")
        if self.goal == None:
            raise(ValueError, "Must create goal node before running search. Try running PathPlanner.set_goal(start_node)")
        if self.start == None:
            raise(ValueError, "Must create start node before running search. Try running PathPlanner.set_start(start_node)")

        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet()
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()
        self.gScore = self.gScore if self.gScore != None else  self.create_gScore()
        self.fScore = self.fScore if self.fScore != None else  self.create_fScore()
        
        count = 0
#         print(self.start, " -> ", self.goal)

        while not self.is_open_empty():
            current = self.get_current_node()
            print(count, ". ", self.openSet, " -> ", current )
             
            if current == self.goal:
                self.path = self.reconstruct_path(current)
#                 print( count, ". steps, RESULTS ARE: ", self.path )
                return self.path
            else:
                self.openSet.remove(current)
                self.closedSet.add(current)

            for neighbor in self.get_neighbors(current):
                if neighbor in self.closedSet:
                    continue    # Ignore the neighbor which is already evaluated.

                if not neighbor in self.openSet:    # Discover a new node
                    self.openSet.add(neighbor)
                
                # The distance from start to a neighbor
                #the "dist_between" function may vary as per the solution requirements.
                if self.get_tentative_gScore(current, neighbor) >= self.get_gScore(neighbor):
                    continue        # This is not a better path.
                    
                # This path is the best until now. Record it!
                self.record_best_path_to(current, neighbor)
            count += 1
        print("No Path Found")
        self.path = None
        return False

## Your Turn

Implement the following functions to get your search algorithm running smoothly!

### Data Structures

The next few functions requre you to decide on data structures to use - lists, sets, dictionaries, etc. Make sure to think about what would work most efficiently for each of these. Some can be returned as just an empty data structure (see `create_closedSet()` for an example), while others should be initialized with one or more values within.

In [250]:
# Function(s)
def create_closedSet(self):
    """ Creates & returns a data structure that holds a set of evaluated nodes """
    # EXAMPLE: return a data structure suitable to hold the set of nodes already evaluated
    return set()

In [251]:
# TODO: return a data structure suitable to hold the set of currently discovered nodes 
# that are not evaluated yet. Make sure to include the start node.
def create_openSet(self):
    """ Creates & returns a data structure that holds a set of currently discovered nodes 
        that are not evaluated yet."""
    if self.start != None:
        openSet = set()
        openSet.add(self.start)
        return openSet
    
    raise(ValueError, "ERROR: Start node must be created before creating an open set.")

In [252]:
def create_openSet(self):
    """Creates & returns a data structure that holds unevaluated nodes."""
    if self.start != None:
        open_set = set()
        open_set.add(self.start) 
        return open_set
    
    raise(ValueError, "ERROR: Start node must be created before creating an open set.")

In [253]:
# TODO: return a data structure that shows which node can most efficiently be reached from another,
# for each node.
def create_cameFrom(self):
    """Creates & returns a data structure that outputs the ideal node to be reached from another node for every node."""
    came_from = { self.start: [self.start] }
    return came_from

In [254]:
# TODO:  return a data structure that holds the cost of getting from the start node to that node, for each node.
# for each node. The cost of going from start to start is zero. The rest of the node's values should 
# be set to infinity.
def create_gScore(self):
    """Creates & returns a data structure that holds the cost of getting from the start node to the designated node 
       for every node.
       The cost of traveling from start to start is set to zero. The rest of the nodes' value is set to infinity."""
    g_Score = {}
    for element in self.map.intersections:
        g_Score.update( {element : float('inf')})
        
    g_Score[self.start] = 0
    return g_Score

In [255]:
# TODO: return a data structure that holds the total cost of getting from the start node to the goal
# by passing by that node, for each node. That value is partly known, partly heuristic.
# For the first node, that value is completely heuristic. The rest of the node's value should be 
# set to infinity.
def create_fScore(self):
    """Creates & returns a data structure that holds the total cost of getting from the start node to the goal
    by passing by that node for every node. That value is partly known, partly heuristic.
    For the first node, that value is completely heuristic."""
    f_Score = {}
    for element in self.map.intersections:
        f_Score.update( {element : float('inf')})
    f_Score[self.start] = 0
    return f_Score

### Set certain variables

The below functions help set certain variables if they weren't a part of initializating our `PathPlanner` class, or if they need to be changed for anothe reason.

In [256]:
# Method(s)
# Purpose: To set certain variables that weren't initialized within class PathPlanner.
def set_map(self, m_value):
    """Set a map attribute. """
    self._reset(self)
    self.start = None
    self.goal = None
    self.map = m_value

In [257]:
def set_start(self, start):
    """Set a start attribute."""
    self._reset(self)
    self.start = start

In [258]:
def set_goal(self, goal):
    """Set a goal attribute."""
    self._reset(self)
    self.goal = goal

### Get node information

The below functions concern grabbing certain node information. In `is_open_empty`, you are checking whether there are still nodes on the frontier to explore. In `get_current_node()`, you'll want to come up with a way to find the lowest `fScore` of the nodes on the frontier. In `get_neighbors`, you'll need to gather information from the map to find the neighbors of the current node.

In [259]:
# Function(s)
# Purpose: To get information of the node.
def get_current_node(self):
    """ Returns the lowest value of a f(node) in a open set. """
    return min(self.openSet, key=self.fScore.get)

In [260]:
def get_neighbors(self, node):
    """Returns the neighbors of a node. """
    return self.map.roads[node]

In [261]:
def is_open_empty(self):
    """Returns True if the open set is empty or zero numerically. 
       therwise. """
    return len(self.openSet) == 0

### Scores and Costs

Below, you'll get into the main part of the calculation for determining the best path - calculating the various parts of the `fScore`.

In [262]:
# Function(s)
# PurposeL To obtain scores and costs in order to determine the best path.
def get_gScore(self, node):
    """Returns the g_Score of a node."""
    # TODO: Return the g_Score of a node
    return self.gScore[node]

In [263]:
def get_tentative_gScore(self, current, neighbor):
    """Returns the tentative g_Score of a node."""
    # TODO: Return the g_Score of the current node 
    # plus distance from the current node to it's neighbors
    return self.get_gScore(current) + self.distance(current, neighbor)

In [264]:
def calculate_fScore(self, node):
    """Calculates & returns the f_Score of a node."""
    # TODO: Calculates & returns the f_Score of a node. 
    # REMEMBER F = G + H
    return self.gScore[node] + self.distance( self.goal, node )

In [265]:
def distance(self, node1, node2):
    """ Calculates & Returns the Euclidean L2 Distance."""
    # TODO: Compute and return the Euclidean L2 Distance
    x1, y1 = self.map.intersections[node1]
    x2, y2 = self.map.intersections[node2]
    return ( (y2 - y1)**2 + (x2 - x1)**2 ) ** 0.5

In [266]:
def heuristic_cost_estimate(self, node):
    """ Returns the heuristic cost estimate of a node."""
    # TODO: Return the heuristic cost estimate of a node
    return self.fScore[node]

### Recording the best path

Now that you've implemented the various functions on scoring, you can record the best path to a given neighbor node from the current node!

In [267]:
def record_best_path_to(self, current, neighbor):
    """Records the best possible path to a node."""
    # TODO: Record the best path to a node, by updating cameFrom, gScore, and fScore
    
    if current == self.start:
        self.cameFrom[neighbor] = [ self.start, neighbor]
    elif neighbor not in self.cameFrom:
        self.cameFrom.update ( { neighbor : [ element for element in self.cameFrom[current] ] })
        self.cameFrom[neighbor].append(neighbor)
    else:
        self.cameFrom.update ( { neighbor : [ element for element in self.cameFrom[current] ] })
        self.cameFrom[neighbor].append(neighbor)
        
    self.gScore[neighbor] = self.get_gScore(current) + self.distance(current, neighbor)
    self.fScore[neighbor] = self.calculate_fscore(neighbor) 

### Associating your functions with the `PathPlanner` class

To check your implementations, we want to associate all of the above functions back to the `PathPlanner` class. Python makes this easy using the dot notation (i.e. `PathPlanner.myFunction`), and setting them equal to your function implementations. Run the below code cell for this to occur.

*Note*: If you need to make further updates to your functions above, you'll need to re-run this code cell to associate the newly updated function back with the `PathPlanner` class again!

In [268]:
# Associates implemented functions with PathPlanner class
PathPlanner.create_closedSet = create_closedSet
PathPlanner.create_openSet = create_openSet
PathPlanner.create_cameFrom = create_cameFrom
PathPlanner.create_gScore = create_gScore
PathPlanner.create_fScore = create_fScore
PathPlanner.set_map = set_map
PathPlanner.set_start = set_start
PathPlanner.set_goal = set_goal
PathPlanner.is_open_empty = is_open_empty
PathPlanner.get_current_node = get_current_node
PathPlanner.get_neighbors = get_neighbors
PathPlanner.get_gScore = get_gScore
PathPlanner.distance = distance
PathPlanner.get_tentative_gScore = get_tentative_gScore
PathPlanner.heuristic_cost_estimate = heuristic_cost_estimate
PathPlanner.calculate_fscore = calculate_fscore
PathPlanner.record_best_path_to = record_best_path_to

### Preliminary Test

The below is the first test case, just based off of one set of inputs. If some of the functions above aren't implemented yet, or are implemented incorrectly, you likely will get an error from running this cell. Try debugging the error to help you figure out what needs further revision!

In [269]:
# Assign Function to Variable
test_planner = PathPlanner(map_40, 5, 34)
test_path = test_planner.path

# Condition that determines whether the path is computable.
if test_path == [5, 16, 37, 12, 34]:
    print("Code Works!")
else:
    print("ERROR: The Path is invalid:")
    print(test_path)

0 .  {5}  ->  5
1 .  {32, 16, 14}  ->  16
2 .  {32, 37, 14, 30}  ->  37
3 .  {32, 12, 14, 22, 29, 30}  ->  12
4 .  {32, 34, 14, 17, 22, 28, 29, 30, 31}  ->  34
Code Works!


#### Visualize

Once the above code worked for you, let's visualize the results of your algorithm!

In [270]:
# Declare & Initialize Variables
start = 3
goal = 20

# Visualize Path
show_map(map_40, start=start, goal=goal, path=PathPlanner(map_40, start, goal).path)

0 .  {3}  ->  3
1 .  {35, 6, 11, 15, 20}  ->  20


In [271]:
# Declare & Initialize Variables
start = 10
goal = 20

# Visualize Path
show_map(map_40, start=start, goal=goal, path=PathPlanner(map_40, start, goal).path)

0 .  {10}  ->  10
1 .  {13, 17, 18, 24, 25, 26, 27, 31}  ->  25
2 .  {1, 34, 13, 15, 17, 18, 24, 26, 27, 31}  ->  1
3 .  {34, 35, 6, 13, 15, 17, 18, 20, 24, 26, 27, 31}  ->  20


### Testing your Code
If the code below produces no errors, your algorithm is behaving correctly. You are almost ready to submit! Before you submit, go through the following submission checklist:

**Submission Checklist**

1. Does my code pass all tests?
2. Does my code implement `A*` search and not some other search algorithm?
3. Do I use an **admissible heuristic** to direct search efforts towards the goal?
4. Do I use data structures which avoid unnecessarily slow lookups?

When you can answer "yes" to all of these questions, and also have answered the written questions below, submit by pressing the Submit button in the lower right!

In [272]:
# Libraries
from test import test

# Function Call
# Purpose: Test to test if all tests are passed.
test(PathPlanner)

0 .  {5}  ->  5
1 .  {32, 16, 14}  ->  16
2 .  {32, 37, 14, 30}  ->  37
3 .  {32, 12, 14, 22, 29, 30}  ->  12
4 .  {32, 34, 14, 17, 22, 28, 29, 30, 31}  ->  34
0 .  {5}  ->  5
0 .  {8}  ->  8
1 .  {33, 30, 14}  ->  33
2 .  {30, 14}  ->  30
3 .  {16, 14}  ->  14
4 .  {16, 5}  ->  16
5 .  {5, 37}  ->  37
6 .  {5, 22, 12, 29}  ->  12
7 .  {17, 34, 5, 22, 28, 29, 31}  ->  34
8 .  {0, 5, 17, 22, 25, 26, 28, 29, 31}  ->  17
9 .  {0, 1, 5, 10, 15, 18, 22, 25, 26, 28, 29, 31}  ->  10
10 .  {0, 1, 5, 13, 15, 18, 22, 24, 25, 26, 27, 28, 29, 31}  ->  24
All tests pass! Congratulations!


## Questions

**Instructions**  

Answer the following questions in your own words. We do not you expect you to know all of this knowledge on the top of your head. We expect you to do research and ask question. However do not merely copy and paste the answer from a google or stackoverflow. Read the information and understand it first. Then use your own words to explain the answer.

---
How would you explain A-Star to a family member(layman)?

**ANSWER**:
The main function of an A-star search algorithm is for path planning and graph reversal. It first starts from a specific node of a graph and then finds a path to a goal node with the smallest cost. The cost estimation is based on two criterias: the cost of moving from start to the current node and the heuristic that estimates the cost of the cheapest path from node n to the goal node.


---
How does A-Star search algorithm differ from Uniform cost search? What about Best First search?

**ANSWER**:
An A-star search algorithm differs from a Uniform Cost search by its count method. A Uniform cost search only counts the cost from start to the current node and an A-star search algorithm searches are informed.


---
What is a heuristic?

**ANSWER**:
 A heuristic is a function that estimates the cost of the cheapest path from current node to the goal.

---
What is a consistent heuristic?

**ANSWER**:
A consistent heuristic is a function that always estimates less than or equal to the estimated distance from any neighboring vertex to the goal, plus the cost of reaching that neighbor.

---
What is a admissible heuristic? 

**ANSWER**:
An admissible heuristic is a function that never overestimates the cost of reaching the goal.


---
___ admissible heuristic are consistent.

*CHOOSE ONE*
    - All
    - Some
    - None
    
**ANSWER**:
Some.

---
___ Consistent heuristic are admissible.

*CHOOSE ONE*
    - All
    - Some
    - None
    
**ANSWER**:
All.

---